In [55]:
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib
import random
import os
import argparse
from sklearn.metrics import precision_recall_curve

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42 

dnase_preds="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/DNASE_SE/GM12878/nautilus_runs/GM12878_03.06.2022_bias_128_4_1234_0.8_fold_0/dsqtl_preds/snp_scores.tsv"
atac_preds="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/GM12878/nautilus_runs/GM12878_03.01.2022_bias_128_4_1234_0.4_fold_0/dsqtl_preds/snp_scores.tsv"
atac_preds_250m="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/GM12878/subsampling/GM12878_250M/dsqtl_preds/snp_scores.tsv"
atac_preds_100m="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/GM12878/subsampling/GM12878_100M/dsqtl_preds/snp_scores.tsv"
atac_preds_50m="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/GM12878/subsampling/GM12878_50M/dsqtl_preds/snp_scores.tsv"
atac_preds_25m="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/GM12878/subsampling/GM12878_25M/dsqtl_preds/snp_scores.tsv"
atac_preds_5m="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/ATAC_PE/GM12878/subsampling/GM12878_5M/dsqtl_preds/snp_scores.tsv"



In [60]:
labels = pd.read_csv("/mnt/lab_data2/anusri/chrombpnet/results/variant_data/dsqtls/process/counts/modisco_cwm_hits.bed",header=None)
labels["new"] = labels[1]
labels["ll"] = labels[5].str.split(",").str[-1].astype(int)
labels["rsid"] = labels[0] + "_" + labels["new"].astype(str) + "_" + labels[3] + "_" + labels[4]


In [57]:
labels

,0,1,2,3,4,5,6,new,ll,rsid
0,chr13,40984793,40984794,T,C,"13.789686999999999,17.423902,13.78968699999999...",Ahr+Arnt,40984793,1,chr13_40984793_T_C
1,chr13,109921586,109921587,C,T,"4.579978,-0.760316,4.579978,0.760316,0.108275,...",Ahr+Arnt,109921586,1,chr13_109921586_C_T
2,chr15,20475182,20475183,C,G,"0.784986,0.002148,0.784986,0.002148,-0.582721,...",Ahr+Arnt,20475182,1,chr15_20475182_C_G
3,chr21,37284025,37284026,T,C,"-5.55317,-0.164467,5.55317,0.164467,1.96872799...",Ahr+Arnt,37284025,1,chr21_37284025_T_C
4,chr5,5418289,5418290,C,T,"-2.429609,-0.8246530000000001,2.429609,0.82465...",Ahr+Arnt,5418289,1,chr5_5418289_C_T
...,...,...,...,...,...,...,...,...,...,...
19848,chr5,115937000,115937001,A,G,"-1.655029,0.036962,1.655029,0.036962,0.9884780...",ATF2,115937000,-1,chr5_115937000_A_G
19849,chr5,177297361,177297362,C,T,"2.014668,-1.2695809999999998,2.014668,1.269580...",ATF3,177297361,-1,chr5_177297361_C_T
19850,chr16,85527587,85527588,T,C,"0.48692700000000005,1.038381,0.486927000000000...",AP2C,85527587,-1,chr16_85527587_T_C
19851,chr3,154363249,154363250,T,C,"3.9586690000000004,5.038149,3.9586690000000004...",Ascl2,154363249,-1,chr3_154363249_T_C


In [58]:
tf_names = set(labels[labels["ll"]==1][6])

In [59]:
#len(labels[labels["ll"]==1])
labels[labels["ll"]==1].groupby(6).count()

,0,1,2,3,4,5,new,ll,rsid
6,,,,,,,,,
AHR,92,92,92,92,92,92,92,92,92
Ahr+Arnt,369,369,369,369,369,369,369,369,369


In [24]:
def fetch_auprc(preds):
    dsqtls = pd.read_csv(preds,sep="\t", header=0, index_col=False)
    dsqtls[['gkm_SVM','kmer_SVM','abs_gkm_SVM','abs_kmer_SVM','CADD','GWAVA_region','GWAVA_tss','GWAVA_unmatched','GERP','label']] = dsqtls['META_DATA'].str.split(",", expand=True)
    dsqtls["label"] = dsqtls["label"].astype(int)
    dsqtls["rsid"] = dsqtls["CHR"] + "_" + dsqtls["POS0"].astype(str) + "_" + dsqtls["REF"]+ "_" + dsqtls["ALT"]
    merged = dsqtls.merge(labels,on="rsid",how="inner")
    merged["abs_probs_jsd_diff"] = abs(merged["probs_jsd_diff"])
    sorted_merged = merged.sort_values('abs_probs_jsd_diff',ascending=False)
    top_1000 = sorted_merged[0:1000]
    print("total_recall",np.sum(top_1000["label"]==1)*100.0/np.sum(merged["label"]==1))
    for tf_name in tf_names:
        print(tf_name, np.sum(top_1000[top_1000[6]==tf_name]["label"]==1)*100.0/np.sum(merged[merged[6]==tf_name]["label"]==1))

    
    
    

In [25]:
roc_500m = fetch_auprc(atac_preds)
roc_250m = fetch_auprc(atac_preds_250m)
roc_100m = fetch_auprc(atac_preds_100m)
roc_50m = fetch_auprc(atac_preds_50m)
roc_25m = fetch_auprc(atac_preds_25m)
roc_5m = fetch_auprc(atac_preds_5m)
roc_dnase = fetch_auprc(dnase_preds)


total_recall 71.64556962025317
ATF1 69.6969696969697
NFYB 85.0
IRF1 71.24183006535948
BATF 66.66666666666667
total_recall 69.11392405063292
ATF1 69.6969696969697
NFYB 90.0
IRF1 67.6470588235294
BATF 66.66666666666667
total_recall 66.58227848101266
ATF1 68.18181818181819
NFYB 70.0
IRF1 66.01307189542484
BATF 66.66666666666667
total_recall 65.82278481012658
ATF1 60.60606060606061
NFYB 75.0
IRF1 66.33986928104575
BATF 66.66666666666667
total_recall 64.0506329113924
ATF1 57.57575757575758
NFYB 70.0
IRF1 65.0326797385621
BATF 66.66666666666667
total_recall 51.64556962025316
ATF1 51.515151515151516
NFYB 55.0
IRF1 51.30718954248366
BATF 66.66666666666667
total_recall 65.0632911392405
ATF1 68.18181818181819
NFYB 70.0
IRF1 63.72549019607843
BATF 100.0


In [8]:
roc_50m.shape

(28308, 19)

In [25]:
roc_500m

,CHR,POS0,REF,ALT,META_DATA,log_counts_diff,log_probs_diff_abs_sum,probs_jsd_diff,gkm_SVM,kmer_SVM,abs_gkm_SVM,abs_kmer_SVM,CADD,GWAVA_region,GWAVA_tss,GWAVA_unmatched,GERP,label,rsid
0,chr1,846445,A,G,"2.653531,7.938784,2.653531,7.938784,-1.522533,...",-0.341796,-271.065060,-0.109347,2.653531,7.938784,2.653531,7.938784,-1.522533,0.48,0.53,0.75,-0.682,1,chr1_846445_A_G
1,chr1,901457,G,A,"2.821422,6.027648,2.821422,6.027648,1.739152,0...",0.193605,172.312500,0.078106,2.821422,6.027648,2.821422,6.027648,1.739152,0.29,0.36,0.89,2.15,1,chr1_901457_G_A
2,chr1,1176364,T,A,"-7.167236,-10.870018,7.167236,10.870018,-0.091...",0.410028,151.351580,0.065117,-7.167236,-10.870018,7.167236,10.870018,-0.091378,0.42,0.49,0.66,0.451,1,chr1_1176364_T_A
3,chr1,1217274,A,G,"-3.957382,2.387491,3.957382,2.387491,1.127472,...",0.902642,587.303830,0.263455,-3.957382,2.387491,3.957382,2.387491,1.127472,0.34,0.43,0.49,2.43,1,chr1_1217274_A_G
4,chr1,1580437,A,G,"-4.785596,-6.827102,4.785596,6.827102,0.665464...",0.268798,121.878720,0.078786,-4.785596,-6.827102,4.785596,6.827102,0.665464,0.38,0.35,0.86,1.64,1,chr1_1580437_A_G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28303,chr9,140155406,T,C,"0.444036,-1.583811,0.444036,1.583811,-0.606667...",-0.028024,-17.351398,-0.008265,0.444036,-1.583811,0.444036,1.583811,-0.606667,0.19,0.22,0.25,-0.89,-1,chr9_140155406_T_C
28304,chr9,140181120,T,C,"1.363332,1.7031439999999998,1.363332,1.7031439...",-0.021937,-11.805570,-0.005908,1.363332,1.7031439999999998,1.363332,1.7031439999999998,0.489617,0.42,0.14,0.16,0.11800000000000001,-1,chr9_140181120_T_C
28305,chr9,140213800,A,G,"-3.1026279999999997,3.3916160000000004,3.10262...",-0.000695,-13.407722,-0.006468,-3.1026279999999997,3.3916160000000004,3.1026279999999997,3.3916160000000004,-0.7449479999999999,0.23,0.18,0.35,0.0,-1,chr9_140213800_A_G
28306,chr9,140219483,G,C,"-4.628759,-2.0871310000000003,4.628759,2.08713...",0.155082,78.820280,0.037146,-4.628759,-2.0871310000000003,4.628759,2.0871310000000003,-0.809512,0.41,0.27,0.15,0.0,-1,chr9_140219483_G_C


In [35]:
labels

,0,1,2,3,4,5,6,new,rsid
0,chr1,15583639,15583640,A,T,"-1.9856580000000001,-1.805157,1.98565800000000...",IRF1,15583639,chr1_15583639_A_T
1,chr1,19270327,19270328,C,A,"6.316054,8.06739,6.316054,8.06739,-0.58205,0.3...",IRF1,19270327,chr1_19270327_C_A
2,chr1,58394295,58394296,A,C,"8.891873,12.697128999999999,8.891873,12.697128...",IRF1,58394295,chr1_58394295_A_C
3,chr1,205192568,205192569,A,G,"7.980336,10.470765,7.980336,10.470765,0.643806...",IRF1,205192568,chr1_205192568_A_G
4,chr10,5927284,5927285,T,C,"-5.092465,-4.977196,5.092465,4.977196,-1.16630...",IRF1,5927284,chr10_5927284_T_C
...,...,...,...,...,...,...,...,...,...
11420,chr15,90406098,90406099,G,A,"-0.137101,0.762966,0.137101,0.762966,-0.560838...",SPIB,90406098,chr15_90406098_G_A
11421,chr2,85846569,85846570,G,A,"1.808546,1.076921,1.808546,1.076921,0.173165,0...",NFKB,85846569,chr2_85846569_G_A
11422,chr5,32525683,32525684,G,A,"-3.0355119999999998,-3.111011,3.03551199999999...",1_5,32525683,chr5_32525683_G_A
11423,chr5,150574387,150574388,C,T,"1.867836,1.574085,1.867836,1.574085,0.572625,0...",1_6,150574387,chr5_150574387_C_T


In [37]:
sum(roc_500m["rsid"].isin(labels["rsid"]))
#roc_500m["rsid"]

11425

In [38]:
labels["rsid"].shape

(11425,)

In [39]:
roc_500m["rsid"].shape

(28308,)

In [55]:
#(np.sum(["label"][-1000:]==1)*100)/np.sum(merged_500m["label"]==1)


85.0

,CHR,POS0,REF,ALT,META_DATA,log_counts_diff,log_probs_diff_abs_sum,probs_jsd_diff,gkm_SVM,kmer_SVM,...,rsid,0,1,2,3,4,5,6,new,abs_probs_jsd_diff
9151,chr5,10193390,G,A,"2.0325729999999997,4.8784089999999996,2.032572...",-0.002589,-5.581182,-0.002150,2.0325729999999997,4.8784089999999996,...,chr5_10193390_G_A,chr5,10193390,10193391,G,A,"2.0325729999999997,4.8784089999999996,2.032572...",1_5,10193390,0.002150
5691,chr19,3401722,A,T,"0.384457,1.044505,0.384457,1.044505,-0.6209720...",0.000185,7.012571,0.003183,0.384457,1.044505,...,chr19_3401722_A_T,chr19,3401722,3401723,A,T,"0.384457,1.044505,0.384457,1.044505,-0.6209720...",0_34,3401722,0.003183
7385,chr20,56635503,C,T,"-2.23814,0.003934,2.23814,0.003934,0.052370000...",0.007952,9.242591,0.003206,-2.23814,0.003934,...,chr20_56635503_C_T,chr20,56635503,56635504,C,T,"-2.23814,0.003934,2.23814,0.003934,0.052370000...",NFKB.1,56635503,0.003206
7926,chr22,35794600,G,C,"1.420157,-0.22514499999999998,1.420157,0.22514...",0.004495,6.446195,0.003256,1.420157,-0.22514499999999998,...,chr22_35794600_G_C,chr22,35794600,35794601,G,C,"1.420157,-0.22514499999999998,1.420157,0.22514...",1_0,35794600,0.003256
7184,chr20,20620252,T,A,"1.2584110000000002,2.1651279999999997,1.258411...",-0.008399,-7.456667,-0.003259,1.2584110000000002,2.1651279999999997,...,chr20_20620252_T_A,chr20,20620252,20620253,T,A,"1.2584110000000002,2.1651279999999997,1.258411...",SP1,20620252,0.003259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,chr2,133056730,G,A,"12.937892,4.391598,12.937892,4.391598,0.61406,...",-1.083802,-697.277900,-0.318716,12.937892,4.391598,...,chr2_133056730_G_A,chr2,133056730,133056731,G,A,"12.937892,4.391598,12.937892,4.391598,0.61406,...",IRF1,133056730,0.318716
57,chr11,44753715,G,C,"-18.558214000000003,-11.52287,18.5582140000000...",1.314020,853.863950,0.325849,-18.558214000000003,-11.52287,...,chr11_44753715_G_C,chr11,44753715,44753716,G,C,"-18.558214000000003,-11.52287,18.5582140000000...",IRF1,44753715,0.325849
63,chr11,64790449,C,T,"6.200455000000001,-0.9204690000000001,6.200455...",-0.735320,-759.546200,-0.328186,6.200455000000001,-0.9204690000000001,...,chr11_64790449_C_T,chr11,64790449,64790450,C,T,"6.200455000000001,-0.9204690000000001,6.200455...",IRF1,64790449,0.328186
19,chr1,89324256,C,G,"-12.804547999999999,-0.718062,12.8045479999999...",0.683694,965.949000,0.367727,-12.804547999999999,-0.718062,...,chr1_89324256_C_G,chr1,89324256,89324257,C,G,"-12.804547999999999,-0.718062,12.8045479999999...",IRF1,89324256,0.367727
